## PACK MODELING WITH EMBEDDED GPs
This is a notebook to explain the example of the new pack modeling infrastructure generated as a wrapper to PyBaMM.

First: Cells must be created with IDs, in this initialization process the operating mode and model type can also be set. The IDs will eventually link the cells to the circuit structure but this is not functional yet.

In [14]:
import timeit
import FoKL.FoKLRoutines
from Battery import battery
import matplotlib
from Battery.new_eval import *

CellA = battery.Cell("A", operating_mode="current")
CellB = battery.Cell("B", operating_mode="current")
CellC = battery.Cell("C", operating_mode="current")
CellD = battery.Cell("D", operating_mode="current")

circuit = [["A","C"], ["B","D"]] # 2s2p model

Next FoKL model parameters from a previously built model on the diffusivity are loaded in.

In [15]:
DN_model = FoKL.FoKLRoutines.load("DN_model.fokl")
betas = DN_model.betas
mtx = DN_model.mtx

T_max_liion = 80 + 273.15
T_min_liion = -30 + 273.15

def DN_func(sto, T):
    beta_mean = betas
    
    return np.exp(evaluate_pybamm_clone(beta_mean, mtx, [sto, (T - T_min_liion) / (T_max_liion - T_min_liion)]))

Other parameters of the Battery are then edited and added

In [19]:
CellA.edit_params(["Current function [A]"], ["[input]"])
CellB.edit_params(["Current function [A]"], ["[input]"])
CellC.edit_params(["Current function [A]"], ["[input]"])
CellD.edit_params(["Current function [A]"], ["[input]"])
CellA.edit_params(["Ambient temperature [K]"], [328.15])
CellB.edit_params(["Ambient temperature [K]"], [348.15])
CellC.edit_params(["Ambient temperature [K]"], [288.15])
CellD.edit_params(["Ambient temperature [K]"], [308.15])

CellA.edit_params(["Negative particle diffusivity [m2.s-1]"], [DN_func])


CellA.inputs = {"Current function [A]": 0.5}
CellB.inputs = {"Current function [A]": 0.5}
CellC.inputs = {"Current function [A]": 0.5}
CellD.inputs = {"Current function [A]": 0.5}

The Battery Object is then created with the cells now set and solved

In [20]:
cells = [CellA, CellB, CellC, CellD]

bat = battery.Battery(cells, [], circuit = circuit)

bat.power_func = lambda t: 9-t/120

t1 = timeit.default_timer()
sol = bat.simulate([0.0, 360.0], 5)
t2 = timeit.default_timer() - t1
print(t2)

bat.plot()


 Circuit:
{'Time [s]': array([0.00000e+00, 5.00000e-03, 1.00000e-02, 2.00000e-02, 4.00000e-02,
       8.00000e-02, 1.60000e-01, 3.20000e-01, 6.40000e-01, 1.28000e+00,
       2.56000e+00, 5.00000e+00, 5.00000e+00, 5.00500e+00, 5.01000e+00,
       5.02000e+00, 5.04000e+00, 5.08000e+00, 5.16000e+00, 5.32000e+00,
       5.64000e+00, 6.28000e+00, 7.56000e+00, 1.00000e+01, 1.00000e+01,
       1.00050e+01, 1.00100e+01, 1.00200e+01, 1.00400e+01, 1.00800e+01,
       1.01600e+01, 1.03200e+01, 1.06400e+01, 1.12800e+01, 1.25600e+01,
       1.50000e+01, 1.50000e+01, 1.50050e+01, 1.50100e+01, 1.50200e+01,
       1.50400e+01, 1.50800e+01, 1.51600e+01, 1.53200e+01, 1.56400e+01,
       1.62800e+01, 1.75600e+01, 2.00000e+01, 2.00000e+01, 2.00050e+01,
       2.00100e+01, 2.00200e+01, 2.00400e+01, 2.00800e+01, 2.01600e+01,
       2.03200e+01, 2.06400e+01, 2.12800e+01, 2.25600e+01, 2.50000e+01,
       2.50000e+01, 2.50050e+01, 2.50100e+01, 2.50200e+01, 2.50400e+01,
       2.50800e+01, 2.51600e+01, 2.53200

RuntimeError: dictionary changed size during iteration